In [1]:
from train_utils import *

Using cuda as DEVICE


In [2]:
train_model(DATASET='potsdam', MODEL_TYPE='DeepLabV3', BACKBONE='r101', NUM_EPOCHS=2, LEARNING_RATE=0.01)

Indexing files in potsdam/Patched... 
Found: 	 342 Images 
	 342 Mask
Length of all files: 333
Length of train (266) and test (67): 333
Length of train loader: 133; Length of test loader: 34 with batch size 2

Evaluating Accuracy on Test Set...

	Accuracy: 26.25
Start Training ...
Epoch: 0 | Batch 5 / 133 | Loss: 1.8163
Epoch: 0 | Batch 10 / 133 | Loss: 1.7559
Epoch: 0 | Batch 15 / 133 | Loss: 1.7055
Epoch: 0 | Batch 20 / 133 | Loss: 1.6793
Epoch: 0 | Batch 25 / 133 | Loss: 1.6456
Epoch: 0 | Batch 30 / 133 | Loss: 1.623
Epoch: 0 | Batch 35 / 133 | Loss: 1.601
Epoch: 0 | Batch 40 / 133 | Loss: 1.5826
Epoch: 0 | Batch 45 / 133 | Loss: 1.5616
Epoch: 0 | Batch 50 / 133 | Loss: 1.5446
Epoch: 0 | Batch 55 / 133 | Loss: 1.5303
Epoch: 0 | Batch 60 / 133 | Loss: 1.5096
Epoch: 0 | Batch 65 / 133 | Loss: 1.4962
Epoch: 0 | Batch 70 / 133 | Loss: 1.476
Epoch: 0 | Batch 75 / 133 | Loss: 1.4582
Epoch: 0 | Batch 80 / 133 | Loss: 1.4382
Epoch: 0 | Batch 85 / 133 | Loss: 1.4239
Epoch: 0 | Batch 90 / 133

### Evaluate the Model 

In [ ]:
# Load the model
model = torch.load(config['data']['model_weights']+'/potsdam_FCN.pth.tar')
optimizer = torch.load(config['data']['model_weights']+'/potsdam_FCN_optim.pth.tar')

In [ ]:
# Get all images, targets and predictions for the test set
img_size = 512
all_images = np.zeros([BATCH_SIZE*(len(test_loader)-1), 3, img_size, img_size])
all_labels = np.zeros([BATCH_SIZE*(len(test_loader)-1), img_size, img_size])
all_predictions = np.zeros([BATCH_SIZE*(len(test_loader)-1), img_size, img_size])

for idx, (data, label) in enumerate(test_loader):
    print(f"Processing batch {idx} of {len(test_loader)-1}")
    if idx != len(test_loader)-1:
        all_images[idx*BATCH_SIZE:((idx+1)*BATCH_SIZE), :, :, :] = data
        all_labels[idx*BATCH_SIZE:((idx+1)*BATCH_SIZE), :, :] = label

        # Get model predictions and save them
        model.eval()
        with torch.no_grad():
            data = data.to(DEVICE)
            logits = model(data)['out']
            prob = nn.Softmax(logits).dim
            pred = torch.argmax(prob, dim = 1)
        all_predictions[idx*BATCH_SIZE:((idx+1)*BATCH_SIZE), :, :] = pred.cpu()

In [ ]:
def calculate_label_area(label):
    total_pixel = label.size
    rel = pd.Series([100*label[label==i].size/total_pixel for i in range(NUM_CLASSES)])
    rel.index = Label_classes
    return rel
def plot_areas(label, prediction):
    label_rel = calculate_label_area(label)
    pred_rel = calculate_label_area(prediction)
    df = pd.DataFrame([label_rel, pred_rel], columns=Label_classes, index = ['target', 'prediction']).transpose().reset_index().melt(id_vars='index', var_name='type')
    
    plt.style.use("dark_background")
    fig, ax = plt.subplots()
    sns.barplot(x='index', y='value', hue='type', data=df)
    plt.xlabel("Class")
    plt.ylabel("Relative Surface")
    plt.title("Area of classes in target(left) and prediction(right)")
    plt.xticks(rotation = 90)

    # Change coloring to same colors as in label mask
    class_colors_hex = {k: '#%02x%02x%02x' % v for k, v in idx2rgb.items()}
    col_list= list(class_colors_hex.values())
    col_list = col_list+col_list
    for i, bar in enumerate(ax.patches):
        col = col_list[i]
        bar.set_color(col)
    for container in ax.containers:
        ax.bar_label(container, fmt='%.1f')
    ax.get_legend().remove()
    plt.show()


In [ ]:
# Plot random instance of the test set
idx = random.randint(0, all_images.shape[0]-1)
img = all_images[idx,:,:,:]
label = all_labels[idx,:,:]
prediction = all_predictions[idx, :, :]

fig, ax = plt.subplots(1,4, figsize = (15,4))
ax[0].imshow(ToPILImage()(torch.from_numpy(img)))
ax[0].set_title('Noramlized Input Image')
ax[1].imshow(ToPILImage()(torch.from_numpy(denormalize_image(img))))
ax[1].set_title('Denormalized Input Image')
ax[2].imshow(map_label2rgb(label))
ax[2].set_title('Target')
ax[3].imshow(map_label2rgb(prediction))
ax[3].set_title('Prediction')
plt.suptitle('Random Image and Prediction from Test Set')
plt.show()

In [ ]:
plot_areas(label, prediction)